<a href="https://colab.research.google.com/github/ronniewillaert/Biofabrication-Exercises/blob/main/Chapter1_Exercises_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STUDENTS: Run this cell first!
!pip install plotly seaborn ipywidgets --quiet

from google.colab import output
output.enable_custom_widget_manager()

print("✅ Setup complete! Now run the main exercise cell below.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00
✅ Setup complete! Now run the main exercise cell below.


In [3]:
# Enhanced Hip Replacement Lifetime Analysis - Chapter 1 Exercise 4
"""
=================================================================================
📖 EXERCISE 4: HIP REPLACEMENT LIFETIME ANALYSIS - INTERACTIVE SIMULATION
=================================================================================

🎯 LEARNING OBJECTIVES:
After completing this exercise, you should be able to:
1. Calculate the expected number of revision surgeries for different patient profiles
2. Analyze how implant material choice affects long-term healthcare costs
3. Compare healthcare economics across different countries/healthcare systems
4. Understand how patient demographics influence medical device selection
5. Evaluate the economic case for biofabricated alternatives

📋 BACKGROUND:
Hip replacements are one of the most common orthopedic procedures, with over 450,000
performed annually in the US alone. However, implants have limited lifespans and often
require costly revision surgeries. This creates both patient suffering and enormous
healthcare costs - exactly the problems biofabrication aims to solve.

💡 WHAT YOU'LL EXPLORE:
- How patient age and gender affect revision surgery needs
- Economic differences between implant materials (titanium, ceramic, cobalt-chrome)
- Healthcare cost variations across different countries
- Statistical analysis using Monte Carlo simulation (1,000 virtual patients)

🎛️ HOW TO USE THIS EXERCISE:
1. Run this cell to load the interactive simulation
2. Adjust the sliders to explore different scenarios:
   - Patient Age: When they receive their first implant (40-75 years)
   - Life Expectancy: How long they're expected to live (75-95 years)
   - Gender: Male/Female (affects implant longevity due to activity differences)
   - Implant Material: Different materials have different lifespans and costs
   - Country: Healthcare systems have different cost structures
3. Observe how the charts change with each adjustment
4. Try extreme scenarios (young patient + long life expectancy)
5. Compare premium materials vs standard options
6. Answer the reflection questions at the bottom

🔍 WHAT TO LOOK FOR:
- Which factors most strongly influence revision needs?
- How does material choice affect total lifetime costs?
- Which countries show the most cost-effective healthcare delivery?
- At what point do revision surgeries become economically prohibitive?

Let's begin the simulation!
"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("🧬 Enhanced Hip Replacement Analysis - Chapter 1, Exercise 4")
print("="*70)
print("Loading interactive simulation...")

# Enhanced Hip Replacement Analysis Class
class EnhancedHipAnalyzer:
    def __init__(self):
        # Implant materials with realistic lifespans and costs (years, base_cost_USD)
        self.implant_materials = {
            'Standard Titanium': {'lifetime_mean': 18, 'lifetime_std': 4, 'cost_multiplier': 1.0},
            'Premium Titanium': {'lifetime_mean': 23, 'lifetime_std': 5, 'cost_multiplier': 1.4},
            'Ceramic': {'lifetime_mean': 25, 'lifetime_std': 4, 'cost_multiplier': 1.8},
            'Cobalt-Chrome': {'lifetime_mean': 20, 'lifetime_std': 4, 'cost_multiplier': 1.2},
            'Experimental Bioactive': {'lifetime_mean': 30, 'lifetime_std': 6, 'cost_multiplier': 2.5}
        }

        # Gender differences (women typically have longer implant life due to lower activity)
        self.gender_factors = {
            'Male': {'lifetime_factor': 0.92, 'activity_note': 'Higher activity levels'},
            'Female': {'lifetime_factor': 1.08, 'activity_note': 'Lower impact activities'}
        }

        # Country-specific healthcare costs (base surgery cost in USD)
        self.country_costs = {
            'United States': {'base_cost': 45000, 'revision_multiplier': 1.3},
            'Germany': {'base_cost': 35000, 'revision_multiplier': 1.2},
            'Belgium': {'base_cost': 28000, 'revision_multiplier': 1.15},
            'United Kingdom': {'base_cost': 25000, 'revision_multiplier': 1.1},
            'India': {'base_cost': 8000, 'revision_multiplier': 1.4},
            'Thailand': {'base_cost': 12000, 'revision_multiplier': 1.25}
        }

    def calculate_revisions(self, start_age, life_expectancy, implant_lifetime):
        """Calculate number of revision surgeries needed"""
        remaining_years = life_expectancy - start_age
        if remaining_years <= implant_lifetime:
            return 0  # No revisions needed

        revisions = 0
        current_age = start_age + implant_lifetime

        while current_age < life_expectancy:
            revisions += 1
            current_age += implant_lifetime

        return revisions

    def simulate_patient_outcomes(self, start_age, life_expectancy, gender, material, country, n_patients=1000):
        """Simulate outcomes for multiple patients with enhanced parameters"""

        # Get material parameters
        mat_data = self.implant_materials[material]
        base_lifetime = mat_data['lifetime_mean']
        lifetime_std = mat_data['lifetime_std']
        cost_multiplier = mat_data['cost_multiplier']

        # Apply gender factor
        gender_factor = self.gender_factors[gender]['lifetime_factor']
        adjusted_lifetime = base_lifetime * gender_factor

        # Get country cost data
        country_data = self.country_costs[country]
        base_cost = country_data['base_cost']
        revision_multiplier = country_data['revision_multiplier']

        # Calculate material-adjusted cost
        surgery_cost = base_cost * cost_multiplier

        # Generate random implant lifetimes
        lifetimes = np.random.normal(adjusted_lifetime, lifetime_std, n_patients)
        lifetimes = np.clip(lifetimes, 8, 40)  # Realistic bounds

        results = []
        for lifetime in lifetimes:
            revisions = self.calculate_revisions(start_age, life_expectancy, lifetime)
            total_cost = surgery_cost + (revisions * surgery_cost * revision_multiplier)

            results.append({
                'implant_lifetime': lifetime,
                'revisions_needed': revisions,
                'total_cost': total_cost,
                'annual_cost': total_cost / (life_expectancy - start_age) if life_expectancy > start_age else 0
            })

        return pd.DataFrame(results)

# Create the enhanced interactive widget
print("✅ Analysis tools loaded successfully!")
print("\n🎛️ INTERACTIVE CONTROLS:")
print("Adjust the sliders below to explore different patient scenarios")
print("-" * 70)

analyzer = EnhancedHipAnalyzer()

@widgets.interact(
    start_age=widgets.IntSlider(
        value=55, min=40, max=75, step=5,
        description='👤 Patient Age:',
        style={'description_width': 'initial'}
    ),
    life_expectancy=widgets.IntSlider(
        value=82, min=75, max=95, step=1,
        description='📊 Life Expectancy:',
        style={'description_width': 'initial'}
    ),
    gender=widgets.Dropdown(
        options=['Male', 'Female'],
        value='Male',
        description='⚥ Gender:',
        style={'description_width': 'initial'}
    ),
    material=widgets.Dropdown(
        options=list(analyzer.implant_materials.keys()),
        value='Standard Titanium',
        description='🔧 Implant Material:',
        style={'description_width': 'initial'}
    ),
    country=widgets.Dropdown(
        options=list(analyzer.country_costs.keys()),
        value='United States',
        description='🌍 Country:',
        style={'description_width': 'initial'}
    )
)
def analyze_enhanced_hip_replacement(start_age, life_expectancy, gender, material, country):
    print(f"\n🔬 SIMULATION RESULTS:")
    print(f"Analyzing {gender.lower()} patient, age {start_age}, in {country}")
    print(f"Material: {material}, Life expectancy: {life_expectancy} years")
    print("-" * 60)

    # Run simulation
    results = analyzer.simulate_patient_outcomes(
        start_age, life_expectancy, gender, material, country
    )

    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Plot 1: Distribution of revisions needed
    revision_counts = results['revisions_needed'].value_counts().sort_index()
    axes[0,0].bar(revision_counts.index, revision_counts.values, alpha=0.7, color='steelblue')
    axes[0,0].set_title('Distribution of Revisions Needed\n(1,000 simulated patients)', fontweight='bold')
    axes[0,0].set_xlabel('Number of Revision Surgeries')
    axes[0,0].set_ylabel('Number of Patients')
    axes[0,0].grid(True, alpha=0.3)

    # Plot 2: Total cost distribution
    axes[0,1].hist(results['total_cost']/1000, bins=30, alpha=0.7, color='darkgreen')
    axes[0,1].set_title('Total Lifetime Cost Distribution', fontweight='bold')
    axes[0,1].set_xlabel('Total Cost (thousands USD)')
    axes[0,1].set_ylabel('Number of Patients')
    axes[0,1].grid(True, alpha=0.3)

    # Plot 3: Implant lifetime vs revisions scatter
    axes[0,2].scatter(results['implant_lifetime'], results['revisions_needed'],
                     alpha=0.6, color='darkred', s=20)
    axes[0,2].set_title('Implant Lifetime vs Revisions', fontweight='bold')
    axes[0,2].set_xlabel('Implant Lifetime (years)')
    axes[0,2].set_ylabel('Revisions Needed')
    axes[0,2].grid(True, alpha=0.3)

    # Plot 4: Annual cost analysis
    axes[1,0].hist(results['annual_cost'], bins=25, alpha=0.7, color='purple')
    axes[1,0].set_title('Annual Healthcare Cost\n(Total cost ÷ Remaining years)', fontweight='bold')
    axes[1,0].set_xlabel('Annual Cost (USD)')
    axes[1,0].set_ylabel('Number of Patients')
    axes[1,0].grid(True, alpha=0.3)

    # Plot 5: Cost vs revisions relationship
    axes[1,1].scatter(results['revisions_needed'], results['total_cost']/1000,
                     alpha=0.6, color='orange', s=20)
    axes[1,1].set_title('Revisions vs Total Cost', fontweight='bold')
    axes[1,1].set_xlabel('Number of Revisions')
    axes[1,1].set_ylabel('Total Cost (thousands USD)')
    axes[1,1].grid(True, alpha=0.3)

    # Plot 6: Summary statistics and insights
    axes[1,2].axis('off')

    # Calculate key statistics
    avg_revisions = results['revisions_needed'].mean()
    avg_cost = results['total_cost'].mean()
    avg_annual_cost = results['annual_cost'].mean()
    max_revisions = results['revisions_needed'].max()
    no_revision_pct = (results['revisions_needed'] == 0).mean() * 100
    multiple_revisions_pct = (results['revisions_needed'] >= 2).mean() * 100
    high_cost_pct = (results['total_cost'] > 100000).mean() * 100

    # Material info
    mat_info = analyzer.implant_materials[material]
    gender_info = analyzer.gender_factors[gender]

    summary_text = f"""📊 SUMMARY STATISTICS

👤 Patient Profile:
• {gender} patient, age {start_age}
• Life expectancy: {life_expectancy} years
• {gender_info['activity_note']}

🔧 Implant Details:
• Material: {material}
• Expected lifetime: {mat_info['lifetime_mean']:.0f} years
• Cost multiplier: {mat_info['cost_multiplier']:.1f}x

💰 Economic Analysis:
• Average revisions: {avg_revisions:.1f}
• Average total cost: ${avg_cost:,.0f}
• Average annual cost: ${avg_annual_cost:,.0f}
• Max revisions needed: {max_revisions}

📈 Patient Outcomes:
• No revisions needed: {no_revision_pct:.1f}%
• Multiple revisions: {multiple_revisions_pct:.1f}%
• High cost cases (>$100k): {high_cost_pct:.1f}%"""

    axes[1,2].text(0.05, 0.95, summary_text, fontsize=11, verticalalignment='top',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))

    plt.suptitle(f'Hip Replacement Analysis: {gender} Age {start_age} - {material} in {country}',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Print detailed insights
    print(f"\n📋 DETAILED INSIGHTS:")
    print(f"• Average revisions per patient: {avg_revisions:.2f}")
    print(f"• Total lifetime healthcare cost: ${avg_cost:,.0f}")
    print(f"• Annual healthcare burden: ${avg_annual_cost:,.0f}")
    print(f"• Patients avoiding revisions: {no_revision_pct:.1f}%")
    print(f"• Patients needing multiple revisions: {multiple_revisions_pct:.1f}%")

    # Generate recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if avg_revisions > 2:
        print("⚠️  HIGH REVISION RATE: Consider premium materials or patient counseling")
    if avg_cost > 80000:
        print("💰 HIGH COST BURDEN: Strong economic case for biofabricated alternatives")
    if multiple_revisions_pct > 30:
        print("🏥 SYSTEM STRAIN: Healthcare system under pressure from revision surgeries")
    if no_revision_pct > 70:
        print("✅ GOOD OUTCOMES: Current treatment approach showing positive results")

    # Biofabrication context
    print(f"\n🧬 BIOFABRICATION CONTEXT:")
    print(f"If biofabricated implants could last 50+ years:")
    bio_revisions = analyzer.calculate_revisions(start_age, life_expectancy, 50)
    bio_cost = analyzer.country_costs[country]['base_cost'] * 2.0  # Assume 2x initial cost
    potential_savings = avg_cost - bio_cost
    print(f"• Estimated revisions needed: {bio_revisions}")
    print(f"• Estimated cost: ${bio_cost:,.0f}")
    print(f"• Potential savings per patient: ${potential_savings:,.0f}")
    print(f"• Break-even if biofab costs < {potential_savings/avg_cost*100:.0f}% more than traditional")

print("\n" + "="*70)
print("🎯 Use the interactive controls above to explore different scenarios!")
print("="*70)

# Reflection questions
display(HTML("""
<div style="background-color: #f0f8ff; padding: 15px; border-left: 5px solid #4169e1; margin-top: 20px;">
<h3>🤔 REFLECTION QUESTIONS</h3>
<p>After exploring different scenarios with the interactive simulation, consider these questions:</p>

<ol>
<li><strong>Patient Demographics:</strong> How do age and gender affect revision surgery needs? Why might these differences exist?</li>

<li><strong>Material Economics:</strong> Compare Standard Titanium vs Experimental Bioactive materials. Under what circumstances is the premium worth the cost?</li>

<li><strong>Healthcare Systems:</strong> How do costs vary between countries? What might explain these differences?</li>

<li><strong>Biofabrication Potential:</strong> Based on your simulations, what would be the "target specifications" for a biofabricated hip implant to be economically competitive?</li>

<li><strong>Societal Impact:</strong> If 450,000 hip replacements are performed annually in the US, use your simulation results to estimate the total economic burden of revision surgeries.</li>

<li><strong>Clinical Decision Making:</strong> For a 45-year-old active male patient, which implant material would you recommend and why?</li>
</ol>

<p><strong>💭 Discussion Starter:</strong> How might the economic analysis change if we consider indirect costs like time off work, caregiver burden, and quality of life impacts?</p>
</div>
"""))

print("\n✨ Exercise 4 Complete!")
print("🔄 Try different combinations of parameters to explore edge cases")
print("📚 Ready for the next exercise or chapter!")

🧬 Enhanced Hip Replacement Analysis - Chapter 1, Exercise 4
Loading interactive simulation...
✅ Analysis tools loaded successfully!

🎛️ INTERACTIVE CONTROLS:
Adjust the sliders below to explore different patient scenarios
----------------------------------------------------------------------


interactive(children=(IntSlider(value=55, description='👤 Patient Age:', max=75, min=40, step=5, style=SliderSt…


🎯 Use the interactive controls above to explore different scenarios!



✨ Exercise 4 Complete!
🔄 Try different combinations of parameters to explore edge cases
📚 Ready for the next exercise or chapter!


In [4]:
# Additional Interactive Exercises for Chapter 1
"""
=================================================================================
📦 EXERCISE 5: ORGAN TRANSPORT LOGISTICS SIMULATION
=================================================================================

🎯 LEARNING OBJECTIVES:
After completing this exercise, you should be able to:
1. Understand the complex logistics of organ transplantation
2. Identify bottlenecks in the organ transport chain
3. Analyze how transport efficiency affects organ viability
4. Calculate the economic cost of failed organ transport
5. Evaluate how biofabrication could eliminate transport challenges

📋 BACKGROUND:
Organ transplantation faces critical timing constraints. A donor kidney remains viable
for only 24-36 hours outside the body. The transport chain involves: procurement →
preservation → transport → recipient preparation → transplantation. Each step introduces
delays and risks, leading to organ wastage and patient deaths.

💡 WHAT YOU'LL EXPLORE:
- Realistic transport timing with uncertainty (traffic, weather, coordination delays)
- Success rates under different logistical scenarios
- Economic cost of failed organ transport
- How biofabrication could revolutionize organ availability

🎛️ HOW TO USE:
Adjust sliders to explore different transport scenarios and efficiency improvements.
"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("🚚 Organ Transport Logistics Simulation - Exercise 5")
print("="*60)

class OrganTransportSimulator:
    def __init__(self):
        self.max_viable_times = {
            'Kidney': 36,      # hours
            'Liver': 12,
            'Heart': 6,
            'Lung': 8,
            'Pancreas': 20
        }

        self.transport_steps = [
            'Procurement',      # 2-4 hours
            'Initial Processing', # 1-2 hours
            'Transport to Hub',  # 1-6 hours
            'Hub Processing',    # 0.5-1 hours
            'Final Transport',   # 1-4 hours
            'Recipient Prep'     # 1-3 hours
        ]

        self.organ_values = {
            'Kidney': 442000,    # USD value if successful
            'Liver': 577000,
            'Heart': 1400000,
            'Lung': 561000,
            'Pancreas': 347000
        }

    def simulate_transport_chain(self, organ_type, efficiency_level, weather_conditions, n_simulations=1000):
        max_time = self.max_viable_times[organ_type]
        organ_value = self.organ_values[organ_type]

        results = []

        for _ in range(n_simulations):
            # Base times for each step (hours) with realistic variability
            procurement_time = np.random.gamma(2, 0.8) + 1  # 1-4 hours
            processing_time = np.random.gamma(1.5, 0.4) + 0.5  # 0.5-2 hours
            transport_1 = np.random.gamma(2, 1.5) + 0.5  # 0.5-6 hours
            hub_processing = np.random.gamma(1, 0.3) + 0.3  # 0.3-1 hour
            transport_2 = np.random.gamma(2, 1) + 0.5  # 0.5-4 hours
            recipient_prep = np.random.gamma(1.5, 0.5) + 0.5  # 0.5-3 hours

            # Apply efficiency factor
            efficiency_multiplier = {
                'Poor': 1.4,
                'Standard': 1.0,
                'Good': 0.8,
                'Excellent': 0.65
            }[efficiency_level]

            # Apply weather factor
            weather_multiplier = {
                'Perfect': 1.0,
                'Good': 1.1,
                'Poor': 1.4,
                'Severe': 1.8
            }[weather_conditions]

            # Calculate adjusted times
            transport_1 *= efficiency_multiplier * weather_multiplier
            transport_2 *= efficiency_multiplier * weather_multiplier
            hub_processing *= efficiency_multiplier

            total_time = (procurement_time + processing_time + transport_1 +
                         hub_processing + transport_2 + recipient_prep)

            success = total_time <= max_time
            economic_value = organ_value if success else 0
            time_buffer = max_time - total_time if success else 0

            results.append({
                'procurement_time': procurement_time,
                'processing_time': processing_time,
                'transport_1': transport_1,
                'hub_processing': hub_processing,
                'transport_2': transport_2,
                'recipient_prep': recipient_prep,
                'total_time': total_time,
                'success': success,
                'time_buffer': time_buffer,
                'economic_value': economic_value
            })

        return pd.DataFrame(results)

# Create interactive organ transport simulation
simulator = OrganTransportSimulator()

@widgets.interact(
    organ_type=widgets.Dropdown(
        options=list(simulator.max_viable_times.keys()),
        value='Kidney',
        description='🫀 Organ Type:',
        style={'description_width': 'initial'}
    ),
    efficiency_level=widgets.Dropdown(
        options=['Poor', 'Standard', 'Good', 'Excellent'],
        value='Standard',
        description='⚡ System Efficiency:',
        style={'description_width': 'initial'}
    ),
    weather_conditions=widgets.Dropdown(
        options=['Perfect', 'Good', 'Poor', 'Severe'],
        value='Good',
        description='🌤️ Weather:',
        style={'description_width': 'initial'}
    )
)
def analyze_organ_transport(organ_type, efficiency_level, weather_conditions):
    print(f"\n🔬 TRANSPORT ANALYSIS: {organ_type}")
    print(f"Conditions: {efficiency_level} efficiency, {weather_conditions.lower()} weather")
    print(f"Maximum viable time: {simulator.max_viable_times[organ_type]} hours")
    print("-" * 50)

    # Run simulation
    results = simulator.simulate_transport_chain(organ_type, efficiency_level, weather_conditions)

    # Create visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Plot 1: Transport step breakdown
    step_times = ['procurement_time', 'processing_time', 'transport_1',
                  'hub_processing', 'transport_2', 'recipient_prep']
    step_labels = ['Procurement', 'Processing', 'Transport 1', 'Hub', 'Transport 2', 'Prep']
    avg_times = [results[step].mean() for step in step_times]

    colors = ['red', 'orange', 'blue', 'green', 'blue', 'purple']
    axes[0,0].bar(step_labels, avg_times, color=colors, alpha=0.7)
    axes[0,0].set_title('Average Time by Transport Step', fontweight='bold')
    axes[0,0].set_ylabel('Time (hours)')
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].grid(True, alpha=0.3)

    # Plot 2: Success rate and total time distribution
    success_rate = results['success'].mean() * 100
    axes[0,1].hist(results['total_time'], bins=30, alpha=0.7, color='steelblue', label='All attempts')
    axes[0,1].axvline(simulator.max_viable_times[organ_type], color='red', linestyle='--',
                     linewidth=2, label=f'Max viable ({simulator.max_viable_times[organ_type]}h)')
    axes[0,1].set_title(f'Total Transport Time\nSuccess Rate: {success_rate:.1f}%', fontweight='bold')
    axes[0,1].set_xlabel('Total Time (hours)')
    axes[0,1].set_ylabel('Number of Cases')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)

    # Plot 3: Economic impact
    successful_cases = results[results['success']]
    failed_cases = results[~results['success']]

    economic_data = ['Successful', 'Failed']
    economic_values = [len(successful_cases), len(failed_cases)]
    economic_colors = ['green', 'red']

    axes[0,2].pie(economic_values, labels=economic_data, colors=economic_colors, autopct='%1.1f%%')
    axes[0,2].set_title('Transport Success Rate', fontweight='bold')

    # Plot 4: Time buffer analysis (for successful cases)
    if len(successful_cases) > 0:
        axes[1,0].hist(successful_cases['time_buffer'], bins=20, alpha=0.7, color='darkgreen')
        axes[1,0].set_title('Time Buffer (Successful Cases)', fontweight='bold')
        axes[1,0].set_xlabel('Remaining Time (hours)')
        axes[1,0].set_ylabel('Number of Cases')
        axes[1,0].grid(True, alpha=0.3)
    else:
        axes[1,0].text(0.5, 0.5, 'No Successful\nCases', ha='center', va='center',
                      transform=axes[1,0].transAxes, fontsize=14)
        axes[1,0].set_title('Time Buffer (Successful Cases)', fontweight='bold')

    # Plot 5: Bottleneck analysis
    step_delays = {}
    for step, label in zip(step_times, step_labels):
        step_delays[label] = results[step].std()  # Use standard deviation as delay indicator

    bottleneck_labels = list(step_delays.keys())
    bottleneck_values = list(step_delays.values())

    axes[1,1].barh(bottleneck_labels, bottleneck_values, color='orange', alpha=0.7)
    axes[1,1].set_title('Process Variability\n(Higher = More Unpredictable)', fontweight='bold')
    axes[1,1].set_xlabel('Standard Deviation (hours)')
    axes[1,1].grid(True, alpha=0.3)

    # Plot 6: Summary statistics
    axes[1,2].axis('off')

    total_economic_value = results['economic_value'].sum()
    avg_transport_time = results['total_time'].mean()
    failed_organs = len(failed_cases)
    economic_loss = failed_organs * simulator.organ_values[organ_type]

    summary_text = f"""📊 LOGISTICS SUMMARY

🕐 Timing Analysis:
• Average transport time: {avg_transport_time:.1f} hours
• Success rate: {success_rate:.1f}%
• Failed organs: {failed_organs}/1000
• Avg buffer (successful): {successful_cases['time_buffer'].mean():.1f}h

💰 Economic Impact:
• Total value delivered: ${total_economic_value/1e6:.1f}M
• Economic loss: ${economic_loss/1e6:.1f}M
• Loss percentage: {economic_loss/((economic_loss + total_economic_value))*100:.1f}%

🚨 Bottlenecks:
• Highest variability: {max(step_delays, key=step_delays.get)}
• Transport steps most affected by:
  - Weather: {weather_conditions}
  - System efficiency: {efficiency_level}

🧬 Biofab Advantage:
• Eliminates transport timing constraints
• Could save {failed_organs} organs per 1000 cases
• Potential value: ${economic_loss/1e6:.1f}M per 1000 procedures"""

    axes[1,2].text(0.05, 0.95, summary_text, fontsize=10, verticalalignment='top',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))

    plt.suptitle(f'Organ Transport Analysis: {organ_type} under {efficiency_level} Efficiency',
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Print key insights
    print(f"\n💡 KEY INSIGHTS:")
    print(f"• Transport success rate: {success_rate:.1f}%")
    print(f"• Economic loss from failed transport: ${economic_loss/1e6:.2f}M per 1000 organs")
    print(f"• Average time buffer (successful cases): {successful_cases['time_buffer'].mean():.1f} hours")
    print(f"• Most variable step: {max(step_delays, key=step_delays.get)}")

    if success_rate < 80:
        print("⚠️  LOW SUCCESS RATE: Urgent need for system improvements")
    if economic_loss > 100000000:  # $100M
        print("💰 HIGH ECONOMIC LOSS: Strong case for alternative solutions")

print("\n🎛️ Use the controls above to explore different transport scenarios!")

"""
=================================================================================
🦎 EXERCISE 6: SPECIES REGENERATION COMPARISON
=================================================================================

🎯 LEARNING OBJECTIVES:
After completing this exercise, you should be able to:
1. Compare regenerative capabilities across different species
2. Understand why humans have limited regeneration capacity
3. Identify biological mechanisms that enable regeneration
4. Connect natural regeneration to biofabrication strategies
5. Analyze the evolutionary trade-offs of regeneration vs other traits

📋 BACKGROUND:
Many animals possess remarkable regenerative abilities that humans lack. Salamanders
can regrow entire limbs, zebrafish can regenerate heart tissue, and some species
like hydra show negligible aging. Understanding these differences helps identify
targets for biofabrication and tissue engineering approaches.

💡 WHAT YOU'LL EXPLORE:
- Regenerative capabilities across the animal kingdom
- Relationship between regeneration ability and other biological traits
- Evolutionary trade-offs in regenerative capacity
- How this knowledge informs biofabrication strategies
"""

print("\n" + "="*60)
print("🦎 Species Regeneration Comparison - Exercise 6")
print("="*60)

class RegenerationAnalyzer:
    def __init__(self):
        self.species_data = {
            'Humans': {
                'tissue_regen': 2, 'organ_regen': 1, 'limb_regen': 0, 'neural_regen': 1,
                'lifespan': 80, 'metabolic_rate': 1.0, 'complexity': 10,
                'research_relevance': 10, 'notes': 'Limited to liver, bone remodeling'
            },
            'Laboratory Mouse': {
                'tissue_regen': 3, 'organ_regen': 2, 'limb_regen': 0, 'neural_regen': 2,
                'lifespan': 2, 'metabolic_rate': 7.0, 'complexity': 8,
                'research_relevance': 9, 'notes': 'Standard research model'
            },
            'Zebrafish': {
                'tissue_regen': 6, 'organ_regen': 7, 'limb_regen': 4, 'neural_regen': 8,
                'lifespan': 3, 'metabolic_rate': 4.0, 'complexity': 6,
                'research_relevance': 8, 'notes': 'Heart, fin, brain regeneration'
            },
            'Axolotl': {
                'tissue_regen': 8, 'organ_regen': 8, 'limb_regen': 10, 'neural_regen': 9,
                'lifespan': 15, 'metabolic_rate': 0.3, 'complexity': 7,
                'research_relevance': 9, 'notes': 'Complete limb regeneration'
            },
            'Planarian Worm': {
                'tissue_regen': 10, 'organ_regen': 10, 'limb_regen': 10, 'neural_regen': 10,
                'lifespan': 0.5, 'metabolic_rate': 2.0, 'complexity': 2,
                'research_relevance': 6, 'notes': 'Whole body from fragments'
            },
            'Hydra': {
                'tissue_regen': 10, 'organ_regen': 9, 'limb_regen': 8, 'neural_regen': 7,
                'lifespan': 1000, 'metabolic_rate': 1.5, 'complexity': 1,
                'research_relevance': 5, 'notes': 'Potentially immortal'
            },
            'Starfish': {
                'tissue_regen': 7, 'organ_regen': 8, 'limb_regen': 9, 'neural_regen': 6,
                'lifespan': 35, 'metabolic_rate': 0.5, 'complexity': 3,
                'research_relevance': 4, 'notes': 'Arm and central disc regeneration'
            }
        }

        self.regeneration_categories = ['tissue_regen', 'organ_regen', 'limb_regen', 'neural_regen']
        self.category_labels = ['Tissue', 'Organ', 'Limb', 'Neural']

# Create interactive species comparison
analyzer = RegenerationAnalyzer()

@widgets.interact(
    selected_species=widgets.SelectMultiple(
        options=list(analyzer.species_data.keys()),
        value=['Humans', 'Axolotl', 'Zebrafish'],
        description='🐾 Select Species:',
        style={'description_width': 'initial'},
        layout={'height': '150px'}
    ),
    analysis_type=widgets.Dropdown(
        options=['Regeneration Radar', 'Evolutionary Trade-offs', 'Research Relevance'],
        value='Regeneration Radar',
        description='📊 Analysis Type:',
        style={'description_width': 'initial'}
    )
)
def analyze_species_regeneration(selected_species, analysis_type):
    print(f"\n🔬 REGENERATION ANALYSIS: {analysis_type}")
    print(f"Comparing: {', '.join(selected_species)}")
    print("-" * 50)

    if analysis_type == 'Regeneration Radar':
        # Create radar chart comparison
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8), subplot_kw=dict(projection='polar'))

        angles = np.linspace(0, 2*np.pi, len(analyzer.category_labels), endpoint=False)
        angles = np.concatenate((angles, [angles[0]]))  # Close the plot

        colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']

        for i, species in enumerate(selected_species):
            if species in analyzer.species_data:
                data = analyzer.species_data[species]
                values = [data[cat] for cat in analyzer.regeneration_categories]
                values += [values[0]]  # Close the plot

                ax1.plot(angles, values, 'o-', linewidth=2, label=species, color=colors[i % len(colors)])
                ax1.fill(angles, values, alpha=0.25, color=colors[i % len(colors)])

        ax1.set_xticks(angles[:-1])
        ax1.set_xticklabels(analyzer.category_labels)
        ax1.set_ylim(0, 10)
        ax1.set_title('Regeneration Capabilities Comparison', fontweight='bold', pad=20)
        ax1.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
        ax1.grid(True)

        # Complexity vs Regeneration scatter plot
        ax2.remove()  # Remove polar projection for scatter plot
        ax2 = fig.add_subplot(1, 2, 2)

        for i, species in enumerate(selected_species):
            if species in analyzer.species_data:
                data = analyzer.species_data[species]
                avg_regen = np.mean([data[cat] for cat in analyzer.regeneration_categories])
                complexity = data['complexity']
                ax2.scatter(complexity, avg_regen, s=200, alpha=0.7,
                          color=colors[i % len(colors)], label=species)
                ax2.annotate(species, (complexity, avg_regen), xytext=(5, 5),
                           textcoords='offset points', fontsize=10)

        ax2.set_xlabel('Biological Complexity (1-10)')
        ax2.set_ylabel('Average Regeneration Ability (0-10)')
        ax2.set_title('Complexity vs Regeneration Trade-off', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

    elif analysis_type == 'Evolutionary Trade-offs':
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

        species_names = []
        lifespans = []
        avg_regens = []
        metabolic_rates = []
        complexities = []

        for species in selected_species:
            if species in analyzer.species_data:
                data = analyzer.species_data[species]
                species_names.append(species)
                lifespans.append(data['lifespan'])
                avg_regens.append(np.mean([data[cat] for cat in analyzer.regeneration_categories]))
                metabolic_rates.append(data['metabolic_rate'])
                complexities.append(data['complexity'])

        # Lifespan vs Regeneration
        ax1.scatter(lifespans, avg_regens, s=150, alpha=0.7)
        for i, species in enumerate(species_names):
            ax1.annotate(species, (lifespans[i], avg_regens[i]), xytext=(5, 5),
                        textcoords='offset points', fontsize=9)
        ax1.set_xlabel('Lifespan (years)')
        ax1.set_ylabel('Average Regeneration Ability')
        ax1.set_title('Lifespan vs Regeneration', fontweight='bold')
        ax1.grid(True, alpha=0.3)

        # Metabolic Rate vs Regeneration
        ax2.scatter(metabolic_rates, avg_regens, s=150, alpha=0.7, color='orange')
        for i, species in enumerate(species_names):
            ax2.annotate(species, (metabolic_rates[i], avg_regens[i]), xytext=(5, 5),
                        textcoords='offset points', fontsize=9)
        ax2.set_xlabel('Metabolic Rate (relative to humans)')
        ax2.set_ylabel('Average Regeneration Ability')
        ax2.set_title('Metabolic Rate vs Regeneration', fontweight='bold')
        ax2.grid(True, alpha=0.3)

        # Complexity vs Regeneration (detailed)
        ax3.scatter(complexities, avg_regens, s=150, alpha=0.7, color='green')
        for i, species in enumerate(species_names):
            ax3.annotate(species, (complexities[i], avg_regens[i]), xytext=(5, 5),
                        textcoords='offset points', fontsize=9)
        ax3.set_xlabel('Biological Complexity')
        ax3.set_ylabel('Average Regeneration Ability')
        ax3.set_title('Complexity vs Regeneration Trade-off', fontweight='bold')
        ax3.grid(True, alpha=0.3)

        # Research relevance bar chart
        research_scores = [analyzer.species_data[species]['research_relevance'] for species in species_names]
        bars = ax4.bar(range(len(species_names)), research_scores, alpha=0.7, color='purple')
        ax4.set_xticks(range(len(species_names)))
        ax4.set_xticklabels(species_names, rotation=45, ha='right')
        ax4.set_ylabel('Research Relevance Score')
        ax4.set_title('Research Model Relevance', fontweight='bold')
        ax4.grid(True, alpha=0.3)

        # Add value labels on bars
        for bar, score in zip(bars, research_scores):
            height = bar.get_height()
            ax4.annotate(f'{score}', xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    else:  # Research Relevance
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

        # Research relevance vs regeneration ability
        research_scores = []
        avg_regens = []
        species_names = []

        for species in selected_species:
            if species in analyzer.species_data:
                data = analyzer.species_data[species]
                research_scores.append(data['research_relevance'])
                avg_regens.append(np.mean([data[cat] for cat in analyzer.regeneration_categories]))
                species_names.append(species)

        ax1.scatter(research_scores, avg_regens, s=200, alpha=0.7)
        for i, species in enumerate(species_names):
            ax1.annotate(species, (research_scores[i], avg_regens[i]), xytext=(5, 5),
                        textcoords='offset points', fontsize=10)
        ax1.set_xlabel('Research Relevance Score (1-10)')
        ax1.set_ylabel('Average Regeneration Ability (0-10)')
        ax1.set_title('Research Utility vs Regeneration Ability', fontweight='bold')
        ax1.grid(True, alpha=0.3)

        # Detailed capability breakdown
        x = np.arange(len(analyzer.category_labels))
        width = 0.8 / len(selected_species)

        for i, species in enumerate(selected_species):
            if species in analyzer.species_data:
                data = analyzer.species_data[species]
                values = [data[cat] for cat in analyzer.regeneration_categories]
                ax2.bar(x + i * width, values, width, label=species, alpha=0.8)

        ax2.set_xlabel('Regeneration Type')
        ax2.set_ylabel('Regeneration Ability (0-10)')
        ax2.set_title('Detailed Regeneration Capabilities', fontweight='bold')
        ax2.set_xticks(x + width * (len(selected_species) - 1) / 2)
        ax2.set_xticklabels(analyzer.category_labels)
        ax2.legend()
        ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print detailed insights
    print(f"\n📊 COMPARATIVE ANALYSIS:")
    for species in selected_species:
        if species in analyzer.species_data:
            data = analyzer.species_data[species]
            avg_regen = np.mean([data[cat] for cat in analyzer.regeneration_categories])
            print(f"\n🐾 {species}:")
            print(f"   • Average regeneration: {avg_regen:.1f}/10")
            print(f"   • Lifespan: {data['lifespan']} years")
            print(f"   • Research relevance: {data['research_relevance']}/10")
            print(f"   • Notes: {data['notes']}")

    print(f"\n🧬 BIOFABRICATION INSIGHTS:")
    best_regenerator = max(selected_species,
                          key=lambda x: np.mean([analyzer.species_data[x][cat]
                                               for cat in analyzer.regeneration_categories]))
    print(f"• Best regenerator in selection: {best_regenerator}")
    print(f"• Key research targets: Neural regeneration, organ regeneration")
    print(f"• Biofabrication can overcome human limitations by mimicking mechanisms from high-regeneration species")

print("\n🎛️ Select species and analysis type to explore regeneration patterns!")

"""
=================================================================================
💰 EXERCISE 7: BIOFABRICATION ECONOMIC IMPACT CALCULATOR
=================================================================================

🎯 LEARNING OBJECTIVES:
After completing this exercise, you should be able to:
1. Calculate market-wide economic impact of biofabrication technologies
2. Understand the scale of current healthcare spending on replacement therapies
3. Analyze cost-benefit scenarios for biofabrication adoption
4. Evaluate investment requirements and ROI for biofabrication development
5. Project future market opportunities and challenges

📋 BACKGROUND:
The global market for tissue engineering and regenerative medicine exceeded $13 billion
in 2023 and is projected to reach $43 billion by 2028. Understanding the economic
landscape helps evaluate the potential impact and investment requirements for
biofabrication technologies.

💡 WHAT YOU'LL EXPLORE:
- Current healthcare spending on replacement therapies
- Market penetration scenarios for biofabricated products
- Cost-effectiveness analysis compared to traditional treatments
- Investment requirements and return on investment calculations
"""

print("\n" + "="*60)
print("💰 Biofabrication Economic Impact Calculator - Exercise 7")
print("="*60)

class EconomicImpactCalculator:
    def __init__(self):
        # Current market data (in billions USD, 2024)
        self.current_markets = {
            'Organ Transplant': {'size': 25.8, 'growth_rate': 0.08, 'addressable': 0.4},
            'Orthopedic Implants': {'size': 54.2, 'growth_rate': 0.06, 'addressable': 0.6},
            'Cardiovascular Devices': {'size': 62.1, 'growth_rate': 0.07, 'addressable': 0.3},
            'Prosthetics': {'size': 8.9, 'growth_rate': 0.09, 'addressable': 0.8},
            'Skin Grafts': {'size': 2.1, 'growth_rate': 0.12, 'addressable': 0.9},
            'Drug Testing': {'size': 15.3, 'growth_rate': 0.11, 'addressable': 0.5}
        }

        # Biofabrication cost scenarios (multiplier vs traditional)
        self.cost_scenarios = {
            'Optimistic': 0.7,   # 30% cheaper than traditional
            'Realistic': 1.2,    # 20% more expensive initially
            'Conservative': 1.8, # 80% more expensive
            'Pessimistic': 2.5   # 150% more expensive
        }

        # Timeline scenarios (years to market penetration)
        self.timeline_scenarios = {
            'Aggressive': 5,
            'Moderate': 10,
            'Conservative': 15,
            'Slow': 20
        }

        # Global regions with different healthcare spending
        self.regional_data = {
            'North America': {'healthcare_spend': 4200, 'population': 370, 'adoption_rate': 0.8},
            'Europe': {'healthcare_spend': 3800, 'population': 750, 'adoption_rate': 0.7},
            'Asia-Pacific': {'healthcare_spend': 1200, 'population': 4600, 'adoption_rate': 0.4},
            'Rest of World': {'healthcare_spend': 800, 'population': 2300, 'adoption_rate': 0.2}
        }

# Create economic impact calculator
calculator = EconomicImpactCalculator()

@widgets.interact(
    target_market=widgets.SelectMultiple(
        options=list(calculator.current_markets.keys()),
        value=['Organ Transplant', 'Orthopedic Implants', 'Prosthetics'],
        description='🎯 Target Markets:',
        style={'description_width': 'initial'},
        layout={'height': '120px'}
    ),
    cost_scenario=widgets.Dropdown(
        options=list(calculator.cost_scenarios.keys()),
        value='Realistic',
        description='💵 Cost Scenario:',
        style={'description_width': 'initial'}
    ),
    timeline=widgets.Dropdown(
        options=list(calculator.timeline_scenarios.keys()),
        value='Moderate',
        description='⏰ Timeline:',
        style={'description_width': 'initial'}
    ),
    max_penetration=widgets.FloatSlider(
        value=0.25, min=0.05, max=0.8, step=0.05,
        description='📊 Max Penetration:',
        style={'description_width': 'initial'}
    ),
    rd_investment=widgets.FloatSlider(
        value=10, min=1, max=50, step=1,
        description='🔬 R&D Investment ($B):',
        style={'description_width': 'initial'}
    )
)
def calculate_economic_impact(target_market, cost_scenario, timeline, max_penetration, rd_investment):
    print(f"\n💼 ECONOMIC IMPACT ANALYSIS")
    print(f"Scenario: {cost_scenario} costs, {timeline.lower()} timeline")
    print(f"Markets: {', '.join(target_market)}")
    print(f"Max market penetration: {max_penetration*100:.0f}%")
    print("-" * 60)

    years_to_penetration = calculator.timeline_scenarios[timeline]
    cost_multiplier = calculator.cost_scenarios[cost_scenario]

    # Calculate market projections
    results = {}
    total_addressable_market = 0
    total_biofab_revenue = 0

    for market in target_market:
        if market in calculator.current_markets:
            data = calculator.current_markets[market]
            current_size = data['size']
            growth_rate = data['growth_rate']
            addressable = data['addressable']

            # Project market size at full penetration
            future_size = current_size * (1 + growth_rate) ** years_to_penetration
            addressable_market = future_size * addressable
            biofab_revenue = addressable_market * max_penetration

            total_addressable_market += addressable_market
            total_biofab_revenue += biofab_revenue

            results[market] = {
                'current_size': current_size,
                'future_size': future_size,
                'addressable': addressable_market,
                'biofab_revenue': biofab_revenue
            }

    # Create comprehensive visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

    # Plot 1: Market size comparison
    markets = list(results.keys())
    current_sizes = [results[m]['current_size'] for m in markets]
    future_sizes = [results[m]['future_size'] for m in markets]
    biofab_revenues = [results[m]['biofab_revenue'] for m in markets]

    x = np.arange(len(markets))
    width = 0.25

    ax1.bar(x - width, current_sizes, width, label='Current Market ($B)', alpha=0.8, color='lightblue')
    ax1.bar(x, future_sizes, width, label=f'Projected Market {years_to_penetration}Y ($B)', alpha=0.8, color='blue')
    ax1.bar(x + width, biofab_revenues, width, label='Biofab Revenue ($B)', alpha=0.8, color='darkblue')

    ax1.set_xlabel('Market Segment')
    ax1.set_ylabel('Market Size (Billions USD)')
    ax1.set_title('Market Size Projections', fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(markets, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot 2: Regional adoption analysis
    regions = list(calculator.regional_data.keys())
    regional_revenues = []

    for region in regions:
        regional_data = calculator.regional_data[region]
        population = regional_data['population']  # millions
        healthcare_spend = regional_data['healthcare_spend']  # $ per capita
        adoption_rate = regional_data['adoption_rate']

        # Estimate regional biofab revenue
        regional_market = (population * healthcare_spend * 1e-6)  # Convert to billions
        regional_biofab = regional_market * 0.05 * max_penetration * adoption_rate  # 5% of healthcare for relevant procedures
        regional_revenues.append(regional_biofab)

    ax2.pie(regional_revenues, labels=regions, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Regional Revenue Distribution', fontweight='bold')

    # Plot 3: ROI analysis over time
    years = np.arange(1, years_to_penetration + 5)
    cumulative_investment = rd_investment * (1 + 0.1) ** years  # 10% annual R&D increase

    # Revenue ramp-up (S-curve)
    revenue_timeline = []
    for year in years:
        if year <= years_to_penetration:
            penetration = (year / years_to_penetration) ** 2 * max_penetration  # Quadratic ramp
        else:
            penetration = max_penetration
        revenue_timeline.append(total_biofab_revenue * penetration)

    cumulative_revenue = np.cumsum(revenue_timeline)
    net_value = cumulative_revenue - cumulative_investment

    ax3.plot(years, cumulative_investment, label='Cumulative Investment', color='red', linewidth=2)
    ax3.plot(years, cumulative_revenue, label='Cumulative Revenue', color='green', linewidth=2)
    ax3.plot(years, net_value, label='Net Value', color='blue', linewidth=2)
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Years from Start')
    ax3.set_ylabel('Value (Billions USD)')
    ax3.set_title('ROI Timeline Analysis', fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # Plot 4: Sensitivity analysis
    penetration_rates = np.arange(0.1, 0.6, 0.05)
    roi_values = []

    for pen_rate in penetration_rates:
        revenue = total_addressable_market * pen_rate
        total_investment = rd_investment * 2  # Assume 2x initial investment over development
        roi = (revenue - total_investment) / total_investment * 100
        roi_values.append(roi)

    ax4.plot(penetration_rates * 100, roi_values, linewidth=3, color='purple')
    ax4.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Break-even')
    ax4.axhline(y=20, color='green', linestyle='--', alpha=0.7, label='Target ROI (20%)')
    ax4.set_xlabel('Market Penetration (%)')
    ax4.set_ylabel('Return on Investment (%)')
    ax4.set_title('ROI Sensitivity Analysis', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Calculate key metrics
    breakeven_year = next((i for i, val in enumerate(net_value) if val > 0), None)
    final_roi = (cumulative_revenue[-1] - cumulative_investment[-1]) / cumulative_investment[-1] * 100

    print(f"\n📊 KEY ECONOMIC METRICS:")
    print(f"• Total addressable market: ${total_addressable_market:.1f}B")
    print(f"• Projected biofab revenue: ${total_biofab_revenue:.1f}B")
    print(f"• Market penetration required: {max_penetration*100:.0f}%")
    print(f"• Time to market penetration: {years_to_penetration} years")
    print(f"• Break-even year: {breakeven_year + 1 if breakeven_year else 'Not achieved'}")
    print(f"• Final ROI: {final_roi:.1f}%")

    # Investment recommendations
    print(f"\n💡 INVESTMENT INSIGHTS:")
    if final_roi > 20:
        print("🟢 HIGH RETURN: Strong investment opportunity")
    elif final_roi > 10:
        print("🟡 MODERATE RETURN: Reasonable investment with careful execution")
    else:
        print("🔴 LOW RETURN: High risk, consider alternative strategies")

    if breakeven_year and breakeven_year < 8:
        print("⚡ FAST PAYBACK: Quick return on investment")
    elif breakeven_year and breakeven_year < 15:
        print("⏳ MODERATE PAYBACK: Standard biotech timeline")
    else:
        print("🐌 SLOW PAYBACK: Long development cycle, high risk")

    # Market recommendations
    highest_revenue_market = max(results.keys(), key=lambda x: results[x]['biofab_revenue'])
    print(f"\n🎯 STRATEGIC RECOMMENDATIONS:")
    print(f"• Focus market: {highest_revenue_market}")
    print(f"• Recommended R&D budget: ${rd_investment:.1f}B over {years_to_penetration} years")
    print(f"• Target regions: {regions[np.argmax(regional_revenues)]}")

    if cost_multiplier > 1.5:
        print("⚠️ COST CONCERN: High production costs may limit adoption")
    if max_penetration < 0.15:
        print("📈 GROWTH OPPORTUNITY: Consider strategies to increase market penetration")

print("\n🎛️ Adjust parameters above to explore different economic scenarios!")

print("\n" + "="*70)
print("🎉 CHAPTER 1 INTERACTIVE EXERCISES COMPLETE!")
print("="*70)
print("You have now completed all interactive exercises for Chapter 1:")
print("✅ Exercise 4: Hip Replacement Analysis (Enhanced)")
print("✅ Exercise 5: Organ Transport Logistics")
print("✅ Exercise 6: Species Regeneration Comparison")
print("✅ Exercise 7: Economic Impact Calculator")
print()
print("🎓 These exercises demonstrate the motivation for biofabrication by showing:")
print("• Current limitations in medical replacement strategies")
print("• Economic burden of existing healthcare approaches")
print("• Natural examples of superior regenerative capabilities")
print("• Market opportunities for biofabrication solutions")
print()
print("📚 Ready to proceed to Chapter 2: Scaffold-free Biofabrication!")

🚚 Organ Transport Logistics Simulation - Exercise 5


interactive(children=(Dropdown(description='🫀 Organ Type:', options=('Kidney', 'Liver', 'Heart', 'Lung', 'Panc…


🎛️ Use the controls above to explore different transport scenarios!

🦎 Species Regeneration Comparison - Exercise 6


interactive(children=(SelectMultiple(description='🐾 Select Species:', index=(0, 3, 2), layout=Layout(height='1…


🎛️ Select species and analysis type to explore regeneration patterns!

💰 Biofabrication Economic Impact Calculator - Exercise 7


interactive(children=(SelectMultiple(description='🎯 Target Markets:', index=(0, 1, 3), layout=Layout(height='1…


🎛️ Adjust parameters above to explore different economic scenarios!

🎉 CHAPTER 1 INTERACTIVE EXERCISES COMPLETE!
You have now completed all interactive exercises for Chapter 1:
✅ Exercise 4: Hip Replacement Analysis (Enhanced)
✅ Exercise 5: Organ Transport Logistics
✅ Exercise 6: Species Regeneration Comparison
✅ Exercise 7: Economic Impact Calculator

🎓 These exercises demonstrate the motivation for biofabrication by showing:
• Current limitations in medical replacement strategies
• Economic burden of existing healthcare approaches
• Natural examples of superior regenerative capabilities
• Market opportunities for biofabrication solutions

📚 Ready to proceed to Chapter 2: Scaffold-free Biofabrication!
